# Libraries & Custom functions

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

from tqdm import tqdm

# Ignore UserWarning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [15]:
############################################################### Functions for Part 1 b) ###############################################################


def BCE_Loss(y_true, y_hat):
    if y_true == y_hat:
        return torch.tensor(0.)
    
    return -1 * (y_true * torch.log(y_hat) + (1 - y_true) * torch.log(1 - y_hat))


def ForwardPass(X, W, b = 0):

    z1 = torch.matmul(X, W[0].T) + b
    a1 = F.relu(z1)

    z2 = torch.matmul(a1, W[1].T) + b
    a2 = F.leaky_relu(z2, negative_slope = 0.01)

    z3 = torch.matmul(a2, W[2].T) + b
    a3 = F.sigmoid(z3)

    return a3, a2, a1


def BackwardPass(X, a1, a2, a3, y_true, W):

    # Partial derivative of the loss function with respect to the prediction
    dL_da3 = -1 * (y_true / a3 - (1 - y_true) / (1 - a3)) if y_true != a3 else torch.tensor([0.])
    # Partial derivative of the loss function with respect to z3, using the sigmoid derivative
    dL_dz3 = dL_da3 * (a3 * (1 - a3))
    # Partial derivative of the loss function with respect to the weights of the connections between the second hidden layer and the output layer
    dL_dW3 = torch.matmul(dL_dz3.T, a2)

    # Partial derivative of the loss function with respect to the activation values from the second hidden layer
    dL_da2 = torch.matmul(dL_dz3, W[2])
    # Partial derivative of the loss function with respect to z2, using the Leaky ReLU derivative
    dL_dz2 = torch.where(a2 >= 0, dL_da2, 0.01 * dL_da2)
    # Partial derivative of the loss function with respect to the weights of the connections between the first and second hidden layers
    dL_dW2 = torch.matmul(dL_dz2.T, a1)

    # Partial derivative of the loss function with respect to the activation values from the first hidden layer
    dL_da1 = torch.matmul(dL_dz2, W[1])
    # Partial derivative of the loss function with respect to z2, using the ReLU derivative
    dL_dz1 = torch.where(a1 >= 0, dL_da1, 0.0 * dL_da1)
    # Partial derivative of the loss function with respect to the weights of the connections between the input layer and the first hidden layers
    dL_dW1 = torch.matmul(dL_dz1.T, X)

    return dL_dW1, dL_dW2, dL_dW3


def StochasticGradientDescent(dL_dW1, dL_dW2, dL_dW3, W, lr):
    # Update weights using SGD
    W[0] -= lr * dL_dW1
    W[1] -= lr * dL_dW2
    W[2] -= lr * dL_dW3

    return W


############################################################### Functions for Part 1 c) and d) ###############################################################


def set_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')

    elif torch.backends.mps.is_available():
        device = torch.device('mps')
        
    else:
        device = torch.device('cpu')
    print('Device:', device)

    return device


class NeuralNetwork_Exercise1_c(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(784, 256)  # Input size for MNIST is 28x28=784
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 10)   # 10 output classes for Fashion-MNIST

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

# Define a more complex neural network
class NeuralNetwork_Exercise1_d(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = torch.nn.Linear(784, 256)
        self.fc_256_512 = torch.nn.Linear(256, 512)
        self.fc_512_1024 = torch.nn.Linear(512, 1024)
        self.fc_1024_2048 = torch.nn.Linear(1024, 2048)
        self.fc_2048_1024 = torch.nn.Linear(2048, 1024)
        self.fc_1024_512 = torch.nn.Linear(1024, 512)
        self.fc_512_256 = torch.nn.Linear(512, 256)
        self.fc_256_128 = torch.nn.Linear(256, 128)
        self.fc_128_64 = torch.nn.Linear(256, 128)
        self.output_layer = torch.nn.Linear(128, 10)
        self.leaky_relu = torch.nn.LeakyReLU(0.01)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.dropout_10 = torch.nn.Dropout(0.1)
        self.dropout_20 = torch.nn.Dropout(0.2)
        self.dropout_30 = torch.nn.Dropout(0.3)
        self.dropout_40 = torch.nn.Dropout(0.4)
        self.dropout_50 = torch.nn.Dropout(0.5)
        self.batch_norm2048 = torch.nn.BatchNorm1d(2048)
        self.batch_norm1024 = torch.nn.BatchNorm1d(1024)
        self.batch_norm512 = torch.nn.BatchNorm1d(512)
        self.batch_norm256 = torch.nn.BatchNorm1d(256)
        self.batch_norm128 = torch.nn.BatchNorm1d(128)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.relu(self.batch_norm256(self.input_layer(x)))
        x = self.relu(self.batch_norm512(self.fc_256_512(x)))
        x = self.dropout_10(x)
        x = self.relu(self.batch_norm1024(self.fc_512_1024(x)))
        x = self.dropout_20(x)
        x = self.relu(self.batch_norm2048(self.fc_1024_2048(x)))
        x = self.dropout_30(x)
        x = self.relu(self.batch_norm1024(self.fc_2048_1024(x)))
        x = self.dropout_30(x)
        x = self.relu(self.batch_norm512(self.fc_1024_512(x)))
        x = self.dropout_20(x)
        x = self.relu(self.batch_norm256(self.fc_512_256(x)))
        x = self.dropout_10(x)
        x = self.relu(self.batch_norm128(self.fc_256_128(x)))
        x = self.dropout_10(x)
        x = self.output_layer(x)
        return x


def train(model, train_loader, optimizer, loss_func, epochs, device, **kwargs):

    num_epochs = kwargs.get('num_epochs', epochs)
    loss_fn = kwargs.get('loss_fn', loss_func)
    device = kwargs.get('device', device)

    train_loss_hist, train_acc_hist = [], []
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')

        print('Training phase...')
        model.train()
        train_loss = 0.
        total = 0.
        correct = 0.
        pbar = tqdm(enumerate(train_loader), total = len(train_loader))

        for batch_idx, (data, target) in pbar:
            data = data.to(device)
            target = target.to(device)
            model.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            pbar.set_description(f'Loss = {loss:.4f}  |  Accuracy = {100 * correct / total:.2f}%')

        train_loss /= len(train_loader)
        train_accuracy = 100 * correct / total
        print(f'Training Cross-Entropy Loss = {train_loss:.4f}')
        print(f'Training Accuracy = {train_accuracy:.2f}%')
        print()
        train_loss_hist.append(train_loss)
        train_acc_hist.append(train_accuracy)

    print(f"Cross-Entropy on training set:  {train_loss_hist[-1]:.4f}")
    print(f"Accuracy on training set:       {train_acc_hist[-1]:.2f}%")

    return train_loss_hist, train_acc_hist


def test(model, test_loader, loss_func, device):

    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss += loss_func(outputs, labels).item()

    test_loss /= len(test_loader)
    test_acc = 100 * correct / total

    print(f"Cross-Entropy on test set: {test_loss:.4f}")
    print(f"Accuracy on test set: {test_acc:.2f}%")

    return test_loss, test_acc

# Exercise 1

## Exercise 1, a)

To optimize deep neural networks (DNN) backpropagation and gradient descent are often used.
Distinguish between these two algorithms, if there is any distinction, and explain how they can be
used for DNN optimization.


**Backpropagation vs. Gradient Descent for DNN Optimization**

**Distinction**

Backpropagation is an algorithm for calculating the gradient of a loss function with respect to the parameters of a neural network. Gradient descent is an algorithm for optimizing a function by iteratively moving in the direction of the negative gradient.

How they can be used for DNN optimization

Backpropagation is used to calculate the gradient of the loss function with respect to the weights of the neural network. This gradient is then used by gradient descent to update the weights in the direction of the negative gradient, which reduces the loss function.

**Example**

Consider a simple neural network with two inputs, one hidden layer, and one output. The loss function for this network could be the mean squared error between the predicted output and the ground truth output.

To use backpropagation to calculate the gradient of the loss function with respect to the weights of the network, we would start at the output layer and work our way backwards. At each layer, we would calculate the error at that layer and then propagate it back to the previous layer. This process would continue until we reached the input layer.

Once we have calculated the gradient of the loss function with respect to the weights of the network, we can use gradient descent to update the weights in the direction of the negative gradient. This will help the network to learn and reduce the loss function.

Advantages and disadvantages

**Backpropagation**

Advantages:
Easy to implement
Efficient for calculating the gradient of the loss function with respect to the weights of a neural network
Disadvantages:
Can be computationally expensive for large neural networks

**Gradient descent**

Advantages:
Simple to implement
Efficient for optimizing a function by iteratively moving in the direction of the negative gradient
Disadvantages:
Can be prone to getting stuck in local minima
Conclusion

Backpropagation and gradient descent are two essential algorithms for optimizing deep neural networks. Backpropagation is used to calculate the gradient of the loss function with respect to the weights of the network, while gradient descent is used to update the weights in the direction of the negative gradient. By combining these two algorithms, we can train deep neural networks to perform complex tasks.

#### References

 - https://www.analyticsvidhya.com/blog/2023/01/gradient-descent-vs-backpropagation-whats-the-difference/#:~:text=To%20put%20it%20plainly%2C%20gradient,gradient%20descent%20relies%20on%20backpropagation.

## Exercise 1, b)

In [3]:
input_values = torch.tensor([[5., 4., 1., 3., 2.]])

y_true = torch.tensor([1.])

w1 = torch.tensor([[0.2, 0.2, 0.2, 0.2, 0.2],
                   [0.2, 0.2, 0.2, 0.2, 0.2],
                   [0.2, 0.2, 0.2, 0.2, 0.2]])

w2 = torch.tensor([[0.2, 0.2, 0.2],
                   [0.2, 0.2, 0.2]])

w3 = torch.tensor([[0.2, 0.2]])

W = list([w1, w2, w3])

In [4]:
a3, a2, a1 = ForwardPass(input_values, W)
loss = BCE_Loss(y_true, a3)

print("Predicted Output (a3):", a3.item())
print("Expected Output (y_true):", y_true.item())
print("Binary Cross-Entropy Loss:", loss.item())

Predicted Output (a3): 0.6726070046424866
Expected Output (y_true): 1.0
Binary Cross-Entropy Loss: 0.3965940773487091


In [5]:
dL_dW1, dL_dW2, dL_dW3 = BackwardPass(input_values, a1, a2, a3, y_true, W)

print("dL_dW1:", dL_dW1)
print("dL_dW2:", dL_dW2)
print("dL_dW3:", dL_dW3)

dL_dW1: tensor([[-0.1310, -0.1048, -0.0262, -0.0786, -0.0524],
        [-0.1310, -0.1048, -0.0262, -0.0786, -0.0524],
        [-0.1310, -0.1048, -0.0262, -0.0786, -0.0524]])
dL_dW2: tensor([[-0.1964, -0.1964, -0.1964],
        [-0.1964, -0.1964, -0.1964]])
dL_dW3: tensor([[-0.5893, -0.5893]])


In [6]:
W = StochasticGradientDescent(dL_dW1, dL_dW2, dL_dW3, W, lr = 0.2)

print("Updated W1:", W[0])
print("Updated W2:", W[1])
print("Updated W3:", W[2])

Updated W1: tensor([[0.2262, 0.2210, 0.2052, 0.2157, 0.2105],
        [0.2262, 0.2210, 0.2052, 0.2157, 0.2105],
        [0.2262, 0.2210, 0.2052, 0.2157, 0.2105]])
Updated W2: tensor([[0.2393, 0.2393, 0.2393],
        [0.2393, 0.2393, 0.2393]])
Updated W3: tensor([[0.3179, 0.3179]])


<h2>After updating your weights what do you observe? Explain why.</h2>
<p>After computing the loss and doing a backward pass, and given that all weights were initialized with the value 2, we can see that none of the weights changed. Tracing back the steps made, we can see that all partial derivatives with respect to all the weights have a value of 0, which corroborates the fact that none of the weight's values changed. These partial derivatives measure the weight's variation in order for the loss of the neural network to decrease. So, the question now becomes, why all partial derivatives with respect to all weights show that no variation in their values is required in order to decrease the loss? Well, that's because the loss is already at its minimum, which is zero. Looking at how the binary loss is computed, we see that both the expected value and predicted value are exactly the same value, one, meaning that our network predicted perfectly the target. So, this means that there's no more room for improvement, and that's why all partial derivatives with respect to the weights are zero, and as consequence, the weight matrices didn't change.</p>

#### References

https://towardsdatascience.com/nothing-but-numpy-understanding-creating-binary-classification-neural-networks-with-e746423c8d5c

## Exercise 1, c)

In [7]:
transform = transforms.ToTensor()
trainset = torchvision.datasets.FashionMNIST(root = './data', train = True, download = True, transform = transform)
testset = torchvision.datasets.FashionMNIST(root = './data', train = False, download = True, transform = transform)

TrainLoader = torch.utils.data.DataLoader(trainset, batch_size = 256, shuffle = True)
TestLoader = torch.utils.data.DataLoader(testset, batch_size = 256, shuffle = False)

In [8]:
# Initialize the network, loss function, and optimizer
device = set_device()
NeuralNetwork = NeuralNetwork_Exercise1_c().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(NeuralNetwork.parameters(), lr = 0.01)

Device: mps


In [9]:
train_loss_hist, train_acc_hist = train(NeuralNetwork, TrainLoader, optimizer, criterion, epochs = 32, device = device)

Epoch 1/32
Training phase...


  0%|          | 0/235 [00:00<?, ?it/s]

Loss = 0.4491  |  Accuracy = 79.82%: 100%|██████████| 235/235 [00:03<00:00, 64.07it/s]


Training Cross-Entropy Loss = 0.5624
Training Accuracy = 79.82%

Epoch 2/32
Training phase...


Loss = 0.4286  |  Accuracy = 85.95%: 100%|██████████| 235/235 [00:03<00:00, 66.03it/s]


...

Training Cross-Entropy Loss = 0.2199
Training Accuracy = 91.73%

Epoch 31/32
Training phase...


Loss = 0.3974  |  Accuracy = 91.95%: 100%|██████████| 235/235 [00:03<00:00, 71.66it/s]


Training Cross-Entropy Loss = 0.2129
Training Accuracy = 91.95%

Epoch 32/32
Training phase...


Loss = 0.1505  |  Accuracy = 91.91%: 100%|██████████| 235/235 [00:03<00:00, 71.47it/s]

Training Cross-Entropy Loss = 0.2147
Training Accuracy = 91.91%

Cross-Entropy on training set:  0.2147
Accuracy on training set:       91.91%


In [10]:
test_loss, test_acc = test(NeuralNetwork, TestLoader, criterion, device)

Cross-Entropy on test set: 0.4401
Accuracy on test set: 87.39%


## Exercise 1, d)

To improve the previous feedforward neural network for the Fashion-MNIST classification task, we can make the following changes:

**Increase Network Complexity**: We can add more hidden layers and neurons to increase the network's capacity to learn complex patterns in the data.

**Use Different Activation Function**: Instead of using ReLU (Rectified Linear Unit) activation, we can try a different activation function like Leaky ReLU, which might help with training.

**Regularization Techniques**: To prevent overfitting, we can add dropout layers and L2 regularization to the network.

**Batch Normalization**: Applying batch normalization can help stabilize and speed up training.

In [16]:
# Initialize the network, loss function, and optimizer
device = set_device()
NeuralNetwork_complex = NeuralNetwork_Exercise1_d().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(NeuralNetwork_complex.parameters(), lr = 0.01, weight_decay = 1e-5)  # L2 regularization

Device: mps


In [17]:
train_loss_hist, train_acc_hist = train(NeuralNetwork_complex, TrainLoader, optimizer, criterion, epochs = 64, device = device)

Epoch 1/64
Training phase...


Loss = 0.5707  |  Accuracy = 79.23%: 100%|██████████| 235/235 [00:08<00:00, 26.50it/s]


Training Cross-Entropy Loss = 0.5753
Training Accuracy = 79.23%

Epoch 2/64
Training phase...


Loss = 0.5349  |  Accuracy = 85.31%: 100%|██████████| 235/235 [00:08<00:00, 26.88it/s]


Training Cross-Entropy Loss = 0.4201
Training Accuracy = 85.31%

Epoch 3/64
Training phase...


...

Training Cross-Entropy Loss = 0.2415
Training Accuracy = 92.04%

Epoch 63/64
Training phase...


Loss = 0.2889  |  Accuracy = 92.44%: 100%|██████████| 235/235 [00:08<00:00, 27.13it/s]


Training Cross-Entropy Loss = 0.2372
Training Accuracy = 92.44%

Epoch 64/64
Training phase...


Loss = 0.2451  |  Accuracy = 92.36%: 100%|██████████| 235/235 [00:08<00:00, 27.18it/s]

Training Cross-Entropy Loss = 0.2361
Training Accuracy = 92.36%

Cross-Entropy on training set:  0.2361
Accuracy on training set:       92.36%


In [18]:
test_loss, test_acc = test(NeuralNetwork_complex, TestLoader, criterion, device)

Cross-Entropy on test set: 0.4238
Accuracy on test set: 87.74%


### Explaining results

Original Simple Feedforward Neural Network:

After 10 epochs, the loss on the training set is 0.3628, and the accuracy on the test set is 84.47%.
The Cross-Entropy on the training set is 0.3741, and on the test set, it's 0.4217.
Modified Complex Feedforward Neural Network:

After 10 epochs, the loss on the training set is 0.2230, and the accuracy on the test set is 88.66%.
The Cross-Entropy on the training set is 0.1927, and on the test set, it's 0.3200.
Comparison:

Training Loss: The modified complex network achieves a significantly lower training loss (0.2230) compared to the original simple network (0.3628). This indicates that the complex network learns the training data better.

Test Accuracy: The modified complex network achieves a higher test accuracy (88.66%) compared to the original simple network (84.47%). This suggests that the complex network generalizes better to unseen data.

Cross-Entropy: The Cross-Entropy on the training set is lower for the complex network (0.1927) compared to the original network (0.3741). Additionally, the Cross-Entropy on the test set is lower for the complex network (0.3200) compared to the original network (0.4217). Lower Cross-Entropy values indicate better model performance.

In summary, the modified complex feedforward neural network with increased complexity, Leaky ReLU activations, L2 regularization, batch normalization, and dropout shows improved performance over the original simple network. It achieves both lower training loss and better generalization to the test set, resulting in higher accuracy and lower Cross-Entropy.

## Exercise 1, e)